In [38]:
#Convolutional Neural Network solving the PDE, Time and Spatial Discretized and turned
#into an image, the regular input to a CNN using patches to capture the dimensions of the image.
#Also using Weight-Sharing, Gradient Clipping and Learning rates.
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Creating patches of the PDE:
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

# Build a simple CNN
class SimpleCNN:
    #Initialization of the convolutional operation:
    def __init__(self, input_size, output_size):
        self.input_size = input_size
        self.output_size = output_size
        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))
        
    #The loss function:
    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2)
    
    #The forward pass:
    def forward(self, input_data):
        return np.dot(self.weights, input_data) + self.bias

    #The backward pass:
    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = input_data.T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights)
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Weight sharing
        dloss_dweights_shared = np.mean(dloss_dweights, axis=0, keepdims=True)
        dloss_dweights = np.tile(dloss_dweights_shared, (self.weights.shape[0], 1))

        # Update weights and bias
        self.weights -= learning_rate * dloss_dweights
        self.bias -= learning_rate * dloss_dbias

        return loss

# Train the model
model = SimpleCNN(input_size=patch_size**2, output_size=1)

#Setting the number of epochs and learning rate (Adapting over iterations)
epochs = 25
learning_rate = 0.1 / (epoch + 1)

#Scaled data
scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = np.array(predicted_output_normalized)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Comment: Loss is very good after only 2 iterations, w. the best score at the 9th iteration,
#It then diverges and becomes worse over the next 16 iterations. The best score increases by
#30% by the 25th iteration.

Epoch 1/25, Loss: 19.488170019721387
Epoch 2/25, Loss: 0.679838683724943
Epoch 3/25, Loss: 0.6394124469552938
Epoch 4/25, Loss: 0.6083874601037205
Epoch 5/25, Loss: 0.5851365955821669
Epoch 6/25, Loss: 0.5683662219993432
Epoch 7/25, Loss: 0.5570356951402693
Epoch 8/25, Loss: 0.550297820059328
Epoch 9/25, Loss: 0.5474546292781071
Epoch 10/25, Loss: 0.5479243795524879
Epoch 11/25, Loss: 0.5512167916127482
Epoch 12/25, Loss: 0.5569143668395438
Epoch 13/25, Loss: 0.5646582000727851
Epoch 14/25, Loss: 0.5741371316747631
Epoch 15/25, Loss: 0.5850793897210324
Epoch 16/25, Loss: 0.5972460971400335
Epoch 17/25, Loss: 0.6104261820035132
Epoch 18/25, Loss: 0.6244323486900393
Epoch 19/25, Loss: 0.6390978553319215
Epoch 20/25, Loss: 0.6542739074858663
Epoch 21/25, Loss: 0.6698275256072626
Epoch 22/25, Loss: 0.6856397791944416
Epoch 23/25, Loss: 0.7016043066944583
Epoch 24/25, Loss: 0.7176260598208886
Epoch 25/25, Loss: 0.7336202255716335


In [37]:
#Adding dropout:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropout:
    def __init__(self, input_size, output_size, dropout_rate=0.5):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))

    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask

    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2)

    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, input_data) + self.bias

    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = input_data.T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights)
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Weight sharing
        dloss_dweights_shared = np.mean(dloss_dweights, axis=0, keepdims=True)
        dloss_dweights = np.tile(dloss_dweights_shared, (self.weights.shape[0], 1))

        # Update weights and bias
        self.weights -= learning_rate * dloss_dweights
        self.bias -= learning_rate * dloss_dbias

        return loss

# Train the model with dropout
model_with_dropout = SimpleCNNWithDropout(input_size=patch_size**2, output_size=1, dropout_rate=1)

epochs = 25
learning_rate = 0.1 / (epoch + 1)

scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = np.array(predicted_output_normalized)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#The loss is actually a bit worse after adding the dropout, it starts off very good,
#gets better until the 12th iteration, and then starts to increase(diverge) like was
#the case without dropout.

Epoch 1/25, Loss: 1.4394153652638368
Epoch 2/25, Loss: 1.4266099261253322
Epoch 3/25, Loss: 1.3513009930620201
Epoch 4/25, Loss: 1.295086454795722
Epoch 5/25, Loss: 1.2533651906714636
Epoch 6/25, Loss: 1.222711230387821
Epoch 7/25, Loss: 1.2005558932698448
Epoch 8/25, Loss: 1.1849589703036247
Epoch 9/25, Loss: 1.1744435469821068
Epoch 10/25, Loss: 1.1678763749532426
Epoch 11/25, Loss: 1.1643808986515467
Epoch 12/25, Loss: 1.1632737405466187
Epoch 13/25, Loss: 1.1640180794833335
Epoch 14/25, Loss: 1.1661892292817535
Epoch 15/25, Loss: 1.1694490584827055
Epoch 16/25, Loss: 1.1735268426011438
Epoch 17/25, Loss: 1.1782048181538671
Epoch 18/25, Loss: 1.1833071917050904
Epoch 19/25, Loss: 1.188691703119482
Epoch 20/25, Loss: 1.1942430898634908
Epoch 21/25, Loss: 1.1998679767980105
Epoch 22/25, Loss: 1.2054908435530292
Epoch 23/25, Loss: 1.2110508135553657
Epoch 24/25, Loss: 1.2164990752689366
Epoch 25/25, Loss: 1.2217967944425976


In [36]:
#Adding Max pooling:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    def __init__(self, input_size, output_size, dropout_rate=1):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))

    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask

    def max_pooling(self, x, pool_size=2):
        return np.max(x.reshape((x.shape[0] // pool_size, pool_size, x.shape[1])), axis=1)

    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2)

    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias

    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = self.max_pooling(input_data).T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights)
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Weight sharing
        dloss_dweights_shared = np.mean(dloss_dweights, axis=0, keepdims=True)
        dloss_dweights = np.tile(dloss_dweights_shared, (self.weights.shape[0], 1))

        # Update weights and bias
        self.weights -= learning_rate * dloss_dweights
        self.bias -= learning_rate * dloss_dbias

        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=1)

epochs = 25
learning_rate = 0.1 / (epoch + 1)

scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = np.array(predicted_output_normalized)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Adding max pooling yields a significant boost in the loss result right off the bat.
#But it diverges throughout, and in the end does not give a better loss than before.
#The 25th epoch gives a loss 200% worse than the 1st epoch.

Epoch 1/25, Loss: 0.48729598241172123
Epoch 2/25, Loss: 0.5182457411377474
Epoch 3/25, Loss: 0.5509811652657421
Epoch 4/25, Loss: 0.5853691010152491
Epoch 5/25, Loss: 0.6212839100601316
Epoch 6/25, Loss: 0.6586068559128915
Epoch 7/25, Loss: 0.6972255515362321
Epoch 8/25, Loss: 0.7370334620912152
Epoch 9/25, Loss: 0.777929457317894
Epoch 10/25, Loss: 0.8198174085766332
Epoch 11/25, Loss: 0.862605826059851
Epoch 12/25, Loss: 0.9062075321189945
Epoch 13/25, Loss: 0.950539367047202
Epoch 14/25, Loss: 0.9955219240142179
Epoch 15/25, Loss: 1.0410793101737412
Epoch 16/25, Loss: 1.087138931255346
Epoch 17/25, Loss: 1.133631297217651
Epoch 18/25, Loss: 1.1804898467787222
Epoch 19/25, Loss: 1.2276507888551287
Epoch 20/25, Loss: 1.2750529591373092
Epoch 21/25, Loss: 1.3226376902054169
Epoch 22/25, Loss: 1.3703486937490608
Epoch 23/25, Loss: 1.4181319535988193
Epoch 24/25, Loss: 1.4659356284080354
Epoch 25/25, Loss: 1.5137099629404533


In [39]:
#Increasing the Pool size:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    def __init__(self, input_size, output_size, dropout_rate=1):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))

    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask

    def max_pooling(self, x, pool_size=5):
        return np.max(x.reshape((x.shape[0] // pool_size, pool_size, x.shape[1])), axis=1)

    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2)

    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias

    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = self.max_pooling(input_data).T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights)
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Weight sharing
        dloss_dweights_shared = np.mean(dloss_dweights, axis=0, keepdims=True)
        dloss_dweights = np.tile(dloss_dweights_shared, (self.weights.shape[0], 1))

        # Update weights and bias
        self.weights -= learning_rate * dloss_dweights
        self.bias -= learning_rate * dloss_dbias

        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=1)

epochs = 25
learning_rate = 0.1 / (epoch + 1)

scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = np.array(predicted_output_normalized)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Increasing the pooling size, the loss starts off a bit worse, but it does not 
#diverge as quickly as before, and is actually more stable than the smaller pool size.

Epoch 1/25, Loss: 0.749511290228364
Epoch 2/25, Loss: 0.7652322179043924
Epoch 3/25, Loss: 0.7807237224738086
Epoch 4/25, Loss: 0.79593361646902
Epoch 5/25, Loss: 0.8108162241539687
Epoch 6/25, Loss: 0.8253318487444063
Epoch 7/25, Loss: 0.8394462858655248
Epoch 8/25, Loss: 0.853130376964142
Epoch 9/25, Loss: 0.8663595976482916
Epoch 10/25, Loss: 0.8791136768961307
Epoch 11/25, Loss: 0.8913762438292213
Epoch 12/25, Loss: 0.9031344993298939
Epoch 13/25, Loss: 0.9143789102395556
Epoch 14/25, Loss: 0.9251029242324929
Epoch 15/25, Loss: 0.9353027037417223
Epoch 16/25, Loss: 0.9449768775359936
Epoch 17/25, Loss: 0.9541263087241204
Epoch 18/25, Loss: 0.9627538781048467
Epoch 19/25, Loss: 0.9708642818944061
Epoch 20/25, Loss: 0.9784638429577661
Epoch 21/25, Loss: 0.9855603347454017
Epoch 22/25, Loss: 0.9921628172016976
Epoch 23/25, Loss: 0.9982814839641161
Epoch 24/25, Loss: 1.0039275202192726
Epoch 25/25, Loss: 1.0091129706203046
Predicted Output Shape: (1, 1)


In [42]:
#Adding momentum and l2-regularization:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    def __init__(self, input_size, output_size, dropout_rate=1, momentum=1.7, l2_lambda=0.001):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate
        self.momentum = momentum
        self.l2_lambda = l2_lambda

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))

        # Momentum terms for weights and bias
        self.v_weights = np.zeros_like(self.weights)
        self.v_bias = np.zeros_like(self.bias)

    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask

    def max_pooling(self, x, pool_size=2):
        pool_height = x.shape[0] // pool_size
        pool_width = pool_size
        return np.max(x[:pool_height * pool_size, :].reshape((pool_height, pool_width, x.shape[1])), axis=1)

    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2) + 0.5 * self.l2_lambda * np.sum(self.weights ** 2)

    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias

    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = self.max_pooling(input_data).T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights) + self.l2_lambda * self.weights
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Update momentum terms
        self.v_weights = self.momentum * self.v_weights - learning_rate * dloss_dweights
        self.v_bias = self.momentum * self.v_bias - learning_rate * dloss_dbias

        # Update weights and bias
        self.weights += self.v_weights
        self.bias += self.v_bias

        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=0.5, momentum=0.9, l2_lambda=0.001)

epochs = 25
learning_rate = 0.1 / (epoch + 1)

scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = model_with_dropout_and_max_pooling.forward(test_input, training=False)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#A dimension error here, but it is not relevant for the calculation of loss.
#This time, the results are not necessarily better than the previous result,
#but after adding momentum and l2-regularization, the loss is more stable,
#and it converges, like I want it to. The mean loss over the 25 iterations is however equal to the previous run.

Epoch 1/25, Loss: 0.97251672624296
Epoch 2/25, Loss: 0.9703780722589356
Epoch 3/25, Loss: 0.9682743475779226
Epoch 4/25, Loss: 0.9662065141956158
Epoch 5/25, Loss: 0.9641753289297903
Epoch 6/25, Loss: 0.9621813533250158
Epoch 7/25, Loss: 0.9602249634050388
Epoch 8/25, Loss: 0.9583063592595606
Epoch 9/25, Loss: 0.9564255744532689
Epoch 10/25, Loss: 0.9545824852467795
Epoch 11/25, Loss: 0.9527768196202789
Epoch 12/25, Loss: 0.951008166091551
Epoch 13/25, Loss: 0.949275982321705
Epoch 14/25, Loss: 0.9475796035028147
Epoch 15/25, Loss: 0.9459182505225302
Epoch 16/25, Loss: 0.9442910379014717
Epoch 17/25, Loss: 0.9426969815006442
Epoch 18/25, Loss: 0.9411350059964566
Epoch 19/25, Loss: 0.9396039521215985
Epoch 20/25, Loss: 0.9381025836709352
Epoch 21/25, Loss: 0.9366295942723087
Epoch 22/25, Loss: 0.9351836139221827
Epoch 23/25, Loss: 0.933763215287155
Epoch 24/25, Loss: 0.9323669197726809
Epoch 25/25, Loss: 0.9309932033603368


ValueError: shapes (1,25) and (12,1) not aligned: 25 (dim 1) != 12 (dim 0)

In [41]:
#Increasing the momentum:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    def __init__(self, input_size, output_size, dropout_rate=1, momentum=1.7, l2_lambda=0.001):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate
        self.momentum = momentum
        self.l2_lambda = l2_lambda

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))

        # Momentum terms for weights and bias
        self.v_weights = np.zeros_like(self.weights)
        self.v_bias = np.zeros_like(self.bias)

    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask

    def max_pooling(self, x, pool_size=2):
        return np.max(x.reshape((x.shape[0] // pool_size, pool_size, x.shape[1])), axis=1)

    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2) + 0.5 * self.l2_lambda * np.sum(self.weights ** 2)

    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias

    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = self.max_pooling(input_data).T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights) + self.l2_lambda * self.weights
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Update momentum terms
        self.v_weights = self.momentum * self.v_weights - learning_rate * dloss_dweights
        self.v_bias = self.momentum * self.v_bias - learning_rate * dloss_dbias

        # Update weights and bias
        self.weights += self.v_weights
        self.bias += self.v_bias

        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=0.5, momentum=0.9, l2_lambda=0.001)

epochs = 25
learning_rate = 0.1 / (epoch + 1)

scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = model_with_dropout_and_max_pooling.forward(test_input, training=False)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Again the dimension error is irrelevant for the loss computation. Increasing the momentum
#did not improve the Loss, it made it 10% worse.

Epoch 1/25, Loss: 1.0291087828494012
Epoch 2/25, Loss: 1.0272175113295872
Epoch 3/25, Loss: 1.0252411260393353
Epoch 4/25, Loss: 1.0231888614179032
Epoch 5/25, Loss: 1.0210695338293556
Epoch 6/25, Loss: 1.0188915439409618
Epoch 7/25, Loss: 1.0166628801691322
Epoch 8/25, Loss: 1.0143911230926639
Epoch 9/25, Loss: 1.0120834507388925
Epoch 10/25, Loss: 1.0097466446550698
Epoch 11/25, Loss: 1.0073870966816147
Epoch 12/25, Loss: 1.0050108163504383
Epoch 13/25, Loss: 1.0026234388355
Epoch 14/25, Loss: 1.000230233388299
Epoch 15/25, Loss: 0.9978361121946466
Epoch 16/25, Loss: 0.995445639594898
Epoch 17/25, Loss: 0.9930630416117012
Epoch 18/25, Loss: 0.9906922157350581
Epoch 19/25, Loss: 0.9883367409171913
Epoch 20/25, Loss: 0.98599988773353
Epoch 21/25, Loss: 0.9836846286689237
Epoch 22/25, Loss: 0.9813936484916685
Epoch 23/25, Loss: 0.9791293546810003
Epoch 24/25, Loss: 0.9768938878756134
Epoch 25/25, Loss: 0.9746891323145056


ValueError: cannot reshape array of size 25 into shape (12,2,1)

In [43]:
#Decreasing the Pool size:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    def __init__(self, input_size, output_size, dropout_rate=1):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))
    
    #Implementation of dropout:
    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask
    
    #The max pooling convolution:
    def max_pooling(self, x, pool_size=1):
        return np.max(x.reshape((x.shape[0] // pool_size, pool_size, x.shape[1])), axis=1)
    
    #Loss function:
    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2)
        
    #Calculation using Dot product
    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias
    
    #The backward pass:
    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = self.max_pooling(input_data).T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights)
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Weight sharing
        dloss_dweights_shared = np.mean(dloss_dweights, axis=0, keepdims=True)
        dloss_dweights = np.tile(dloss_dweights_shared, (self.weights.shape[0], 1))

        # Update weights and bias
        self.weights -= learning_rate * dloss_dweights
        self.bias -= learning_rate * dloss_dbias

        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=1)

#Set the no. of epochs and the learning rate:
epochs = 25
learning_rate = 0.1 / (epoch + 1)

scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = np.array(predicted_output_normalized)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Decreasing the pool size to 1 again yielded more stable results, similar to the addition of 
#momentum and l2-regularization. It now converges.

Epoch 1/25, Loss: 0.9296405022162872
Epoch 2/25, Loss: 0.9283072180730906
Epoch 3/25, Loss: 0.9269917233876318
Epoch 4/25, Loss: 0.9256923662786471
Epoch 5/25, Loss: 0.9244074752468087
Epoch 6/25, Loss: 0.9231353636812872
Epoch 7/25, Loss: 0.9218743341566021
Epoch 8/25, Loss: 0.9206226825235528
Epoch 9/25, Loss: 0.9193787017988466
Epoch 10/25, Loss: 0.918140685857332
Epoch 11/25, Loss: 0.9169069329316383
Epoch 12/25, Loss: 0.91567574892357
Epoch 13/25, Loss: 0.9144454505320273
Epoch 14/25, Loss: 0.9132143682021762
Epoch 15/25, Loss: 0.9119808489005875
Epoch 16/25, Loss: 0.9107432587211063
Epoch 17/25, Loss: 0.9094999853265869
Epoch 18/25, Loss: 0.9082494402310476
Epoch 19/25, Loss: 0.9069900609270625
Epoch 20/25, Loss: 0.9057203128636879
Epoch 21/25, Loss: 0.9044386912792378
Epoch 22/25, Loss: 0.9031437228940355
Epoch 23/25, Loss: 0.9018339674678587
Epoch 24/25, Loss: 0.9005080192267513
Epoch 25/25, Loss: 0.8991645081638854


In [48]:
#Using 3 max pooling layers/filter instead of 1, pool size = 2:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    #Initialization:
    def __init__(self, input_size, output_size, dropout_rate=1):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))
        
    #Dropout function:
    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask
    
    #Max Pooling functionality:
    def max_pooling(self, x, pool_size=2):
        pool_height = x.shape[0] // pool_size
        pool_width = pool_size

        # First max pooling layer
        x_pooled = np.max(x[:pool_height * pool_size, :].reshape((pool_height, pool_width, x.shape[1])), axis=1)

        # Second max pooling layer
        pool_height = x_pooled.shape[0] // pool_size
        x_pooled = np.max(x_pooled[:pool_height * pool_size, :].reshape((pool_height, pool_width, x_pooled.shape[1])), axis=1)

        # Third max pooling layer
        pool_height = x_pooled.shape[0] // pool_size
        x_pooled = np.max(x_pooled[:pool_height * pool_size, :].reshape((pool_height, pool_width, x_pooled.shape[1])), axis=1)
        
        #Return layers within the function.
        return x_pooled
    
    #The loss function
    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2)

    #The forward pass
    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask
        
        #Calculation using Dot product, a mathematically expensive operation and can be difficult for huge datasets
        #(Can be improved for instance, 
        #by using inception layers)
        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias
    
    #The backward pass
    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias for the dense layer
        dpredictions_dweights_2 = x_pooled_flat.T
        dpredictions_dbias_2 = 1.0

        # Gradients of loss with respect to weights and bias for the dense layer
        dloss_dweights_2 = np.dot(dloss_dpredictions, dpredictions_dweights_2)
        dloss_dbias_2 = np.sum(dloss_dpredictions * dpredictions_dbias_2)

        # Update weights and bias for the dense layer
        self.weights_2 -= learning_rate * dloss_dweights_2
        self.bias_2 -= learning_rate * dloss_dbias_2
        
        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=1)

#No. of epochs/iterations and learning rate(the learning rate changes along the epochs)
epochs = 25
learning_rate = 0.1 / (epoch + 1)

#Scaled data:
scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop(Using the patches for the images(Transformed PDE)):
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = np.array(predicted_output_normalized)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Adding 2 max pooling layers to the previous 1 max pooling layer greatly improved the loss,
#it is stable, and it also converges, providing the best result yet.

Epoch 1/25, Loss: 0.600433786756424
Epoch 2/25, Loss: 0.5963356805645813
Epoch 3/25, Loss: 0.5922357284472378
Epoch 4/25, Loss: 0.5881345516859566
Epoch 5/25, Loss: 0.5840327647775346
Epoch 6/25, Loss: 0.5799309752660632
Epoch 7/25, Loss: 0.5758297835840581
Epoch 8/25, Loss: 0.5717297829026802
Epoch 9/25, Loss: 0.5676315589908633
Epoch 10/25, Loss: 0.5635356900828887
Epoch 11/25, Loss: 0.559442746754543
Epoch 12/25, Loss: 0.5553532918073663
Epoch 13/25, Loss: 0.5512678801610933
Epoch 14/25, Loss: 0.5471870587537181
Epoch 15/25, Loss: 0.5431113664493721
Epoch 16/25, Loss: 0.5390413339535266
Epoch 17/25, Loss: 0.5349774837354803
Epoch 18/25, Loss: 0.5309203299579555
Epoch 19/25, Loss: 0.5268703784134005
Epoch 20/25, Loss: 0.522828126467152
Epoch 21/25, Loss: 0.5187940630069393
Epoch 22/25, Loss: 0.5147686683988082
Epoch 23/25, Loss: 0.5107524144490162
Epoch 24/25, Loss: 0.5067457643720673
Epoch 25/25, Loss: 0.5027491727642819


In [106]:
#Greatly decreasing the l2-regularization:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    def __init__(self, input_size, output_size, dropout_rate=1, momentum=0.9, l2_lambda=0.000001):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate
        self.momentum = momentum
        self.l2_lambda = l2_lambda

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))

        # Momentum terms for weights and bias
        self.v_weights = np.zeros_like(self.weights)
        self.v_bias = np.zeros_like(self.bias)

    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask

    def max_pooling(self, x, pool_size=2):
        pool_height = x.shape[0] // pool_size
        pool_width = pool_size
        return np.max(x[:pool_height * pool_size, :].reshape((pool_height, pool_width, x.shape[1])), axis=1)

    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2) + 0.5 * self.l2_lambda * np.sum(self.weights ** 2)

    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias

    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = self.max_pooling(input_data).T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights) + self.l2_lambda * self.weights
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Update momentum terms
        self.v_weights = self.momentum * self.v_weights - learning_rate * dloss_dweights
        self.v_bias = self.momentum * self.v_bias - learning_rate * dloss_dbias

        # Update weights and bias
        self.weights += self.v_weights
        self.bias += self.v_bias

        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=0.5, momentum=0.9, l2_lambda=0.000001)

epochs = 25
learning_rate = 0.1 / (epoch + 1)

scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = model_with_dropout_and_max_pooling.forward(test_input, training=False)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Decreasing the l2-regularization, the loss is very bad,
#and it does not converge strictly, like I want it to. It does however converge in a sense, but the end result loss
#is not good and it indicates I need a lot of epochs to generate a good result here.

Epoch 1/25, Loss: 98564.44620622172
Epoch 2/25, Loss: 39570.975096831105
Epoch 3/25, Loss: 12134.772058319404
Epoch 4/25, Loss: 9649.025408429445
Epoch 5/25, Loss: 10894.058208341317
Epoch 6/25, Loss: 11604.287538487282
Epoch 7/25, Loss: 11353.166929911938
Epoch 8/25, Loss: 10436.131660783958
Epoch 9/25, Loss: 9221.865531259788
Epoch 10/25, Loss: 8163.630310351647
Epoch 11/25, Loss: 7258.14443503094
Epoch 12/25, Loss: 6455.5541170265715
Epoch 13/25, Loss: 5723.898785736836
Epoch 14/25, Loss: 4964.957139805589
Epoch 15/25, Loss: 4133.616252927568
Epoch 16/25, Loss: 3455.054988284962
Epoch 17/25, Loss: 2945.080779672834
Epoch 18/25, Loss: 2502.182429920468
Epoch 19/25, Loss: 2141.5176662126746
Epoch 20/25, Loss: 1850.3241020615844
Epoch 21/25, Loss: 1608.5537158620928
Epoch 22/25, Loss: 1415.4844551535405
Epoch 23/25, Loss: 1265.577545298806
Epoch 24/25, Loss: 1147.6629304151238
Epoch 25/25, Loss: 1063.8762502199884


ValueError: shapes (1,25) and (12,1) not aligned: 25 (dim 1) != 12 (dim 0)

In [107]:
#Greatly increasing the l2-regularization:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    def __init__(self, input_size, output_size, dropout_rate=1, momentum=0.9, l2_lambda=1):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate
        self.momentum = momentum
        self.l2_lambda = l2_lambda

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))

        # Momentum terms for weights and bias
        self.v_weights = np.zeros_like(self.weights)
        self.v_bias = np.zeros_like(self.bias)

    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask

    def max_pooling(self, x, pool_size=2):
        pool_height = x.shape[0] // pool_size
        pool_width = pool_size
        return np.max(x[:pool_height * pool_size, :].reshape((pool_height, pool_width, x.shape[1])), axis=1)

    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2) + 0.5 * self.l2_lambda * np.sum(self.weights ** 2)

    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias

    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = self.max_pooling(input_data).T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights) + self.l2_lambda * self.weights
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Update momentum terms
        self.v_weights = self.momentum * self.v_weights - learning_rate * dloss_dweights
        self.v_bias = self.momentum * self.v_bias - learning_rate * dloss_dbias

        # Update weights and bias
        self.weights += self.v_weights
        self.bias += self.v_bias

        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=0.5, momentum=0.9, l2_lambda=1)

epochs = 25
learning_rate = 0.1 / (epoch + 1)

scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = model_with_dropout_and_max_pooling.forward(test_input, training=False)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Increasing the l2-regularization, the loss is very bad,
#and it does not converge strictly, like I want it to. 

Epoch 1/25, Loss: 1005.2374234536366
Epoch 2/25, Loss: 967.3832537982488
Epoch 3/25, Loss: 945.4262134185143
Epoch 4/25, Loss: 932.9281987451906
Epoch 5/25, Loss: 932.2274856857196
Epoch 6/25, Loss: 940.1565740918371
Epoch 7/25, Loss: 954.1817479693354
Epoch 8/25, Loss: 974.2492770814231
Epoch 9/25, Loss: 998.7311759692318
Epoch 10/25, Loss: 1022.8571783356274
Epoch 11/25, Loss: 1045.8708758070413
Epoch 12/25, Loss: 1068.029642581275
Epoch 13/25, Loss: 1091.0016945262291
Epoch 14/25, Loss: 1115.4298857043357
Epoch 15/25, Loss: 1139.7372746373844
Epoch 16/25, Loss: 1165.719535417762
Epoch 17/25, Loss: 1197.2019460716003
Epoch 18/25, Loss: 1221.2320093860035
Epoch 19/25, Loss: 1235.9366488636279
Epoch 20/25, Loss: 1244.1468261945095
Epoch 21/25, Loss: 1246.3998375665246
Epoch 22/25, Loss: 1244.2529111985098
Epoch 23/25, Loss: 1238.4417828534029
Epoch 24/25, Loss: 1230.1825531981187
Epoch 25/25, Loss: 1219.758326992487


ValueError: shapes (1,25) and (12,1) not aligned: 25 (dim 1) != 12 (dim 0)

In [108]:
#Greatly increasing the l2-regularization:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    for i in range(1, Nx-1):
        u[i, j+1] = u[i, j] + dt / (2 * dx**2) * (u[i+1, j] - 2*u[i, j] + u[i-1, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    def __init__(self, input_size, output_size, dropout_rate=1, momentum=0.9, l2_lambda=0.001):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate
        self.momentum = momentum
        self.l2_lambda = l2_lambda

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))

        # Momentum terms for weights and bias
        self.v_weights = np.zeros_like(self.weights)
        self.v_bias = np.zeros_like(self.bias)

    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask

    def max_pooling(self, x, pool_size=2):
        pool_height = x.shape[0] // pool_size
        pool_width = pool_size
        return np.max(x[:pool_height * pool_size, :].reshape((pool_height, pool_width, x.shape[1])), axis=1)

    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2) + 0.5 * self.l2_lambda * np.sum(self.weights ** 2)

    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias

    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias
        dpredictions_dweights = self.max_pooling(input_data).T
        dpredictions_dbias = 1.0

        # Gradients of loss with respect to weights and bias
        dloss_dweights = np.dot(dloss_dpredictions, dpredictions_dweights) + self.l2_lambda * self.weights
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        # Gradient clipping
        grad_clip = 10.0
        dloss_dweights = np.clip(dloss_dweights, -grad_clip, grad_clip)
        dloss_dbias = np.clip(dloss_dbias, -grad_clip, grad_clip)

        # Update momentum terms
        self.v_weights = self.momentum * self.v_weights - learning_rate * dloss_dweights
        self.v_bias = self.momentum * self.v_bias - learning_rate * dloss_dbias

        # Update weights and bias
        self.weights += self.v_weights
        self.bias += self.v_bias

        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=0.5, momentum=0.9, l2_lambda=0.001)

epochs = 25
learning_rate = 0.1 / (epoch + 1)

scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data (replace this with your actual test data)
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = model_with_dropout_and_max_pooling.forward(test_input, training=False)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Increasing the l2-regularization, the loss is very bad,
#and it does not converge strictly, like I want it to. 

Epoch 1/25, Loss: 1208.088075130057
Epoch 2/25, Loss: 1195.6533056604505
Epoch 3/25, Loss: 1182.8458152437122
Epoch 4/25, Loss: 1169.9976510971687
Epoch 5/25, Loss: 1157.1996003295371
Epoch 6/25, Loss: 1144.5912353351662
Epoch 7/25, Loss: 1132.1301236550828
Epoch 8/25, Loss: 1119.8875592177794
Epoch 9/25, Loss: 1107.8895488001372
Epoch 10/25, Loss: 1096.2993079596483
Epoch 11/25, Loss: 1085.349352564321
Epoch 12/25, Loss: 1074.7951719984633
Epoch 13/25, Loss: 1064.382785171043
Epoch 14/25, Loss: 1055.5450811964201
Epoch 15/25, Loss: 1047.391072306418
Epoch 16/25, Loss: 1039.7697020053254
Epoch 17/25, Loss: 1032.5592401912597
Epoch 18/25, Loss: 1025.670122028968
Epoch 19/25, Loss: 1019.3520714396892
Epoch 20/25, Loss: 1013.4411985020475
Epoch 21/25, Loss: 1007.9268046849604
Epoch 22/25, Loss: 1002.7581169558804
Epoch 23/25, Loss: 997.9302629118462
Epoch 24/25, Loss: 993.4143791987491
Epoch 25/25, Loss: 989.5264526295529


ValueError: shapes (1,25) and (12,1) not aligned: 25 (dim 1) != 12 (dim 0)

In [111]:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

for j in range(0, Nt):
    # Solve using Implicit Euler's Method
    A = np.eye(Nx) + dt / (dx**2) * (-2 * np.eye(Nx, k=0) + np.eye(Nx, k=1) + np.eye(Nx, k=-1))
    A[0, 0] = 1.0
    A[-1, -1] = 1.0

    u[:, j+1] = np.linalg.solve(A, u[:, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)

class SimpleCNNWithDropoutAndMaxPooling:
    def __init__(self, input_size, output_size, dropout_rate=1):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))

    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask

    def max_pooling(self, x, pool_size=2):
        pool_height = x.shape[0] // pool_size
        pool_width = pool_size

        x_pooled = np.max(x[:pool_height * pool_size, :].reshape((pool_height, pool_width, x.shape[1])), axis=1)

        pool_height = x_pooled.shape[0] // pool_size
        x_pooled = np.max(x_pooled[:pool_height * pool_size, :].reshape((pool_height, pool_width, x_pooled.shape[1])), axis=1)

        pool_height = x_pooled.shape[0] // pool_size
        x_pooled = np.max(x_pooled[:pool_height * pool_size, :].reshape((pool_height, pool_width, x_pooled.shape[1])), axis=1)

        return x_pooled

    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2)

    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask

        return np.dot(self.weights, self.max_pooling(input_data, pool_size=2).flatten().reshape(-1, 1)) + self.bias

    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)
        dpredictions_dweights = self.max_pooling(input_data, pool_size=2).flatten()
        dpredictions_dbias = 1.0

        dloss_dweights = np.outer(dloss_dpredictions, dpredictions_dweights)
        dloss_dbias = np.sum(dloss_dpredictions * dpredictions_dbias)

        self.weights -= learning_rate * dloss_dweights
        self.bias -= learning_rate * dloss_dbias

        return loss
    
# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=1)

# No. of epochs/iterations and learning rate (the learning rate changes along the epochs)
epochs = 25
learning_rate = 0.1 / (epoch + 1)

# Scaled data
scaler_input = StandardScaler()
scaler_output = StandardScaler()
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop (Using the patches for the images(Transformed PDE)):
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]

        loss = model_with_dropout_and_max_pooling.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = model_with_dropout_and_max_pooling.forward(test_input, training=False)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

C:\Users\Kjæreng\AppData\Local\Temp\ipykernel_6180\1061678538.py:72: RuntimeWarning: divide by zero encountered in divide
  mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)


ValueError: shapes (1,25) and (3,1) not aligned: 25 (dim 1) != 3 (dim 0)

In [ ]:
#Using 3 max pooling layers/filter instead of 1, pool size = 2:
import numpy as np
from sklearn.preprocessing import StandardScaler

# Constants
L = 1.0
T = 1.0
Nx = 100  # Number of spatial points
Nt = 1000  # Number of time steps
dx = L / (Nx - 1)
dt = T / (10 * Nt)

# Initial condition
def initial_condition(x):
    return np.sin(np.pi * x)

# Boundary conditions
def boundary_conditions(u):
    u[0] = 0.0
    u[-1] = 0.0

# Generate training data
x_values = np.linspace(0, L, Nx)
t_values = np.linspace(0, T, Nt+1)

u_init = initial_condition(x_values)
u = np.zeros((Nx, Nt+1))
u[:, 0] = u_init

alpha = 0.1  # Adjust the alpha value as needed

for j in range(0, Nt):
    u[1:-1, j + 1] = u[1:-1, j] + alpha * (u[2:, j] - 2 * u[1:-1, j] + u[:-2, j])

    # Apply boundary conditions
    boundary_conditions(u[:, j+1])

# Create image-like patches
input_data = []
output_data = []

patch_size = 5  # Adjust the patch size as needed

for j in range(Nt):
    for i in range(Nx - patch_size + 1):
        input_patch = u[i:i+patch_size, j:j+patch_size].flatten()
        output_patch = u[i+patch_size//2, j+patch_size//2] if i+patch_size//2 < Nx and j+patch_size//2 < Nt else 0.0

        # Pad smaller patches
        if len(input_patch) < patch_size**2:
            input_patch = np.pad(input_patch, (0, patch_size**2 - len(input_patch)))

        input_data.append(input_patch)
        output_data.append(output_patch)

input_data = np.array(input_data)
output_data = np.array(output_data).reshape(-1, 1)


class SimpleCNNWithDropoutAndMaxPooling:
    #Initialization:
    def __init__(self, input_size, output_size, dropout_rate=1):
        self.input_size = input_size
        self.output_size = output_size
        self.dropout_rate = dropout_rate

        # Use He/Glorot initialization for weights
        self.weights = np.random.randn(output_size, input_size) * np.sqrt(2 / (input_size + output_size))
        self.bias = np.zeros((output_size, 1))
        
    #Dropout function:
    def dropout(self, x):
        mask = (np.random.rand(*x.shape) < self.dropout_rate) / (1 - self.dropout_rate)
        return x * mask
    
    #Max Pooling functionality:
    def max_pooling(self, x, pool_size=2):
        pool_height = x.shape[0] // pool_size
        pool_width = pool_size

        # First max pooling layer
        x_pooled = np.max(x[:pool_height * pool_size, :].reshape((pool_height, pool_width, x.shape[1])), axis=1)

        # Second max pooling layer
        pool_height = x_pooled.shape[0] // pool_size
        x_pooled = np.max(x_pooled[:pool_height * pool_size, :].reshape((pool_height, pool_width, x_pooled.shape[1])), axis=1)

        # Third max pooling layer
        pool_height = x_pooled.shape[0] // pool_size
        x_pooled = np.max(x_pooled[:pool_height * pool_size, :].reshape((pool_height, pool_width, x_pooled.shape[1])), axis=1)
        
        #Return layers within the function.
        return x_pooled
    
    #The loss function
    def loss(self, predictions, targets):
        return np.mean((predictions - targets) ** 2)

    #The forward pass
    def forward(self, input_data, training=True):
        if training:
            self.mask = self.dropout(np.ones_like(input_data))
            input_data *= self.mask
        
        #Calculation using Dot product, a mathematically expensive operation and can be difficult for huge datasets
        #(Can be improved for instance, 
        #by using inception layers)
        return np.dot(self.weights, self.max_pooling(input_data)) + self.bias
    
    #The backward pass
    def backward(self, input_data, output_data, learning_rate):
        predictions = self.forward(input_data, training=True)
        loss = self.loss(predictions, output_data)

        # Gradient of loss with respect to predictions
        dloss_dpredictions = 2 * (predictions - output_data) / len(output_data)

        # Gradient of predictions with respect to weights and bias for the dense layer
        dpredictions_dweights_2 = x_pooled_flat.T
        dpredictions_dbias_2 = 1.0

        # Gradients of loss with respect to weights and bias for the dense layer
        dloss_dweights_2 = np.dot(dloss_dpredictions, dpredictions_dweights_2)
        dloss_dbias_2 = np.sum(dloss_dpredictions * dpredictions_dbias_2)

        # Update weights and bias for the dense layer
        self.weights_2 -= learning_rate * dloss_dweights_2
        self.bias_2 -= learning_rate * dloss_dbias_2
        
        return loss

# Train the model with dropout and max pooling
model_with_dropout_and_max_pooling = SimpleCNNWithDropoutAndMaxPooling(input_size=patch_size**2, output_size=1, dropout_rate=1)

#No. of epochs/iterations and learning rate(the learning rate changes along the epochs)
epochs = 25
learning_rate = 0.00001 / (epoch + 1)

#Scaled data:
scaler_input = StandardScaler()
scaler_output = StandardScaler()
# Normalize the input and output data
input_data_normalized = (input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)
output_data_normalized = (output_data - np.mean(output_data)) / np.std(output_data)

# Training loop(Using the patches for the images(Transformed PDE)):
for epoch in range(epochs):
    total_loss = 0
    for i in range(len(input_data_normalized)):
        input_patch = input_data_normalized[i].reshape((patch_size**2, 1))
        output_patch = output_data_normalized[i]
        
        loss = model.backward(input_patch, output_patch, learning_rate)
        total_loss += loss
    
    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss}")

# Generate test data
test_input = input_data[0].reshape((patch_size**2, 1))
predicted_output_normalized = np.array(predicted_output_normalized)

# Denormalize the predicted output
predicted_output = predicted_output_normalized * np.std(output_data) + np.mean(output_data)

#Adding 2 max pooling layers to the previous 1 max pooling layer greatly improved the loss,
#it is stable, and it also converges, providing the best result yet.

Epoch 1/25, Loss: 4012471.4883261328
